In [169]:
api_key=api_key

In [170]:
from apiclient.discovery import build
from datetime import date, timedelta
import csv
import time
t = date.today()
today = t.strftime("%d%m%y")

In [171]:
youtube = build("youtube", "v3", developerKey=api_key)

In [172]:
type(youtube)

googleapiclient.discovery.Resource

In [173]:
res = youtube.channels().list(id='UC3IZKseVpdzPSBaWxBxundA', part='contentDetails').execute()

In [174]:
res

{'kind': 'youtube#channelListResponse',
 'etag': '_p4YEJ0YDMhwuS6KI0ixFX1yxoA',
 'pageInfo': {'resultsPerPage': 1},
 'items': [{'kind': 'youtube#channel',
   'etag': 'GNCgTmtfxb1fB1H-0F3Kq--sb6c',
   'id': 'UC3IZKseVpdzPSBaWxBxundA',
   'contentDetails': {'relatedPlaylists': {'likes': '',
     'favorites': '',
     'uploads': 'UU3IZKseVpdzPSBaWxBxundA',
     'watchHistory': 'HL',
     'watchLater': 'WL'}}}]}

In [175]:
res = youtube.playlistItems().list(playlistId='PL_Cqw69_m_yz4JcOfmZb2IDWwIuej1xfN', part='snippet',maxResults=50).execute()

In [176]:
res = youtube.playlists().list(channelId='UC3IZKseVpdzPSBaWxBxundA', part='snippet',maxResults=50).execute()

In [177]:
res

{'kind': 'youtube#playlistListResponse',
 'etag': 'BIyahLg0SIX-W762ym6Z6q5Mo8E',
 'pageInfo': {'totalResults': 21, 'resultsPerPage': 50},
 'items': [{'kind': 'youtube#playlist',
   'etag': 'IhetKzkt4tZveIqxhxau6uPWcJs',
   'id': 'PL_Cqw69_m_yypdKsu2xV0M0UZ0V1OuiXl',
   'snippet': {'publishedAt': '2020-10-21T15:08:57Z',
    'channelId': 'UC3IZKseVpdzPSBaWxBxundA',
    'title': 'ENHYPEN',
    'description': '',
    'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/lnrUJr8B7oI/default.jpg',
      'width': 120,
      'height': 90},
     'medium': {'url': 'https://i.ytimg.com/vi/lnrUJr8B7oI/mqdefault.jpg',
      'width': 320,
      'height': 180},
     'high': {'url': 'https://i.ytimg.com/vi/lnrUJr8B7oI/hqdefault.jpg',
      'width': 480,
      'height': 360},
     'standard': {'url': 'https://i.ytimg.com/vi/lnrUJr8B7oI/sddefault.jpg',
      'width': 640,
      'height': 480},
     'maxres': {'url': 'https://i.ytimg.com/vi/lnrUJr8B7oI/maxresdefault.jpg',
      'width': 1280,
      '

In [178]:
def get_channel_videos(channel_id, playlist_id=None):
    res = youtube.channels().list(id=channel_id, part='contentDetails').execute()
    if playlist_id==None:
        playlist_id = res['items'][0]['contentDetails']['relatedPlaylists']['uploads']
    else:
        print("")
    videos = []
    next_page_token=None
    while 1:
        res=youtube.playlistItems().list(playlistId=playlist_id, part='snippet',maxResults=50,pageToken=next_page_token).execute()
        videos += res['items']
        next_page_token=res.get('nextPageToken')
        if next_page_token is None:
            break
    return videos

In [179]:
videos=get_channel_videos('UCLkAepWjdylmXSltofFvsYQ','PLm15tOgDZruT7M2arkmVwLFITu4MwyaIG')

In [180]:
len(videos)

31

In [181]:
videos[1]['snippet']['title']

"BTS (방탄소년단) 'Dynamite' Official MV (B-side)"

In [182]:
res = youtube.videos().list(id=videos[1]['snippet']['resourceId']['videoId'], part='statistics').execute()

In [183]:
res

{'kind': 'youtube#videoListResponse',
 'etag': 'b2lj3JZCRtbCPV_-n_sAYg4-maM',
 'items': [{'kind': 'youtube#video',
   'etag': 'RmarfTh-_v2bmjnUcSdjcn-6RKo',
   'id': 'BV2FdDmGiW0',
   'statistics': {'viewCount': '88469034',
    'likeCount': '6646109',
    'dislikeCount': '119683',
    'favoriteCount': '0',
    'commentCount': '413380'}}],
 'pageInfo': {'totalResults': 1, 'resultsPerPage': 1}}

In [184]:
def get_videos_stats(video_ids):
    stats = []
    for i in range(0, len(video_ids), 50):
        res = youtube.videos().list(id=','.join(video_ids[i:i+50]),
                                   part='statistics').execute()
        stats += res['items']
    return stats

In [185]:
video_ids = list(map(lambda x:x['snippet']['resourceId']['videoId'], videos))

In [186]:
stats = get_videos_stats(video_ids)

In [187]:
stats[0]

{'kind': 'youtube#video',
 'etag': 'aef0e_IzkH7q_YVBzV1QtU5MOSU',
 'id': 'gdZLi9oWNZg',
 'statistics': {'viewCount': '533020501',
  'likeCount': '20275796',
  'dislikeCount': '1049074',
  'favoriteCount': '0',
  'commentCount': '8777850'}}

In [188]:
todaysstats={}
counter=0
for video in videos:
    title=video['snippet']['title']
    todaysstats[title]=stats[counter]['statistics']
    counter=counter+1
todaysstats

{"BTS (방탄소년단) 'Dynamite' Official MV": {'viewCount': '533020501',
  'likeCount': '20275796',
  'dislikeCount': '1049074',
  'favoriteCount': '0',
  'commentCount': '8777850'},
 "BTS (방탄소년단) 'Dynamite' Official MV (B-side)": {'viewCount': '88469034',
  'likeCount': '6646109',
  'dislikeCount': '119683',
  'favoriteCount': '0',
  'commentCount': '413380'},
 "BTS (방탄소년단) 'Stay Gold' Official MV": {'viewCount': '129635019',
  'likeCount': '9300784',
  'dislikeCount': '252941',
  'favoriteCount': '0',
  'commentCount': '1785763'},
 "BTS (방탄소년단) 'ON' Official MV": {'viewCount': '204041743',
  'likeCount': '9905290',
  'dislikeCount': '254984',
  'favoriteCount': '0',
  'commentCount': '2840429'},
 "BTS (방탄소년단) 'ON' Kinetic Manifesto Film : Come Prima": {'viewCount': '278423277',
  'likeCount': '10466781',
  'dislikeCount': '291513',
  'favoriteCount': '0',
  'commentCount': '1711388'},
 "BTS (방탄소년단) 'Black Swan' Official MV": {'viewCount': '188016002',
  'likeCount': '8211637',
  'dislikeCou

In [189]:
thecsv=f"/Users/ajkoenig/YouTube/BTS{today}.csv"
with open(thecsv, 'w') as csvfile:
    fieldnames = ['video', 'viewCount', 'likeCount', 'dislikeCount', 'commentCount']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for video in todaysstats:
        for field in fieldnames[1:]:
            try:
                test=todaysstats[video][field]
            except:
                todaysstats[video][field]=None
        writer.writerow({'video': video, 'viewCount':todaysstats[video]['viewCount'],
                         'likeCount':todaysstats[video]['likeCount'],'dislikeCount':todaysstats[video]['dislikeCount'],
                        'commentCount':todaysstats[video]['commentCount']})

In [ ]:
while True:
    videos=get_channel_videos('UC3IZKseVpdzPSBaWxBxundA','PL_Cqw69_m_yz4JcOfmZb2IDWwIuej1xfN')
    res = youtube.videos().list(id=videos[0]['snippet']['resourceId']['videoId'], part='statistics').execute()
    video_ids = list(map(lambda x:x['snippet']['resourceId']['videoId'], videos))
    stats = get_videos_stats(video_ids)
    video="BTS (방탄소년단) 'Dynamite' Official MV"
    title=videos[0]['snippet']['title']
    dynamite={}
    dynamite[title]=stats[0]['statistics']
    print(dynamite)
    time.sleep(60)


{"BTS (방탄소년단) 'Dynamite' Official MV": {'viewCount': '93054091', 'likeCount': '8436490', 'dislikeCount': '245921', 'favoriteCount': '0', 'commentCount': '3302755'}}

{"BTS (방탄소년단) 'Dynamite' Official MV": {'viewCount': '93054091', 'likeCount': '8440042', 'dislikeCount': '246062', 'favoriteCount': '0', 'commentCount': '3303505'}}

{"BTS (방탄소년단) 'Dynamite' Official MV": {'viewCount': '93054091', 'likeCount': '8443628', 'dislikeCount': '246231', 'favoriteCount': '0', 'commentCount': '3294157'}}

{"BTS (방탄소년단) 'Dynamite' Official MV": {'viewCount': '93054091', 'likeCount': '8447242', 'dislikeCount': '246397', 'favoriteCount': '0', 'commentCount': '3308630'}}

{"BTS (방탄소년단) 'Dynamite' Official MV": {'viewCount': '93552595', 'likeCount': '8451016', 'dislikeCount': '246543', 'favoriteCount': '0', 'commentCount': '3305844'}}

{"BTS (방탄소년단) 'Dynamite' Official MV": {'viewCount': '93552595', 'likeCount': '8454538', 'dislikeCount': '246692', 'favoriteCount': '0', 'commentCount': '3306575'}}

{"B


{"BTS (방탄소년단) 'Dynamite' Official MV": {'viewCount': '96005190', 'likeCount': '8638105', 'dislikeCount': '254528', 'favoriteCount': '0', 'commentCount': '3819825'}}

{"BTS (방탄소년단) 'Dynamite' Official MV": {'viewCount': '96005190', 'likeCount': '8643041', 'dislikeCount': '254695', 'favoriteCount': '0', 'commentCount': '3824256'}}

{"BTS (방탄소년단) 'Dynamite' Official MV": {'viewCount': '96005190', 'likeCount': '8648032', 'dislikeCount': '254878', 'favoriteCount': '0', 'commentCount': '3825344'}}

{"BTS (방탄소년단) 'Dynamite' Official MV": {'viewCount': '96333799', 'likeCount': '8652993', 'dislikeCount': '255060', 'favoriteCount': '0', 'commentCount': '3826433'}}

{"BTS (방탄소년단) 'Dynamite' Official MV": {'viewCount': '96333799', 'likeCount': '8658160', 'dislikeCount': '255251', 'favoriteCount': '0', 'commentCount': '3827391'}}

{"BTS (방탄소년단) 'Dynamite' Official MV": {'viewCount': '96333799', 'likeCount': '8663336', 'dislikeCount': '255453', 'favoriteCount': '0', 'commentCount': '3828502'}}

{"B

In [21]:
#def get_videos_stats_dict(video_ids):
    #todaysstats = {}
    #for i in range(0, len(video_ids), 50):
        #res = youtube.videos().list(id=','.join(video_ids[i:i+50]),
                                   #part='statistics').execute()
        #titles = youtube.videos().list(id=','.join(video_ids[i:i+50]),
                                   #part='statistics').execute()
        #todaysstats += res['items']
    #return todaysstats

In [77]:
#get_videos_stats_dict(video_ids)